# Pre Processing EquiRectangular Images and aligning them to a common coordinate system and orientation

### Search Equilocated Sample Frames from the two different datest using KDTree and global coordinate system coordinates.

In [ ]:
import os
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

### Preprocessing using the metadata information

In [ ]:
import json
dec_10 = json.load(open(f"{RAW_DATA_DIR}/Turner-AECOM-Titans Stadium/Week Changes/Dec_10th/metadata/Dec_10th/main_images_full.json", "r"))

In [ ]:
len(dec_10)

In [ ]:
dec_17_1 = json.load(open(f"{RAW_DATA_DIR}/Turner-AECOM-Titans Stadium/Week Changes/Dec_17th/metadata/dec_17/Cap1/main_images_full.json", "r"))
dec_17_2 = json.load(open(f"{RAW_DATA_DIR}/Turner-AECOM-Titans Stadium/Week Changes/Dec_17th/metadata/dec_17/Cap2/main_images_full.json", "r"))
dec_17_3 = json.load(open(f"{RAW_DATA_DIR}/Turner-AECOM-Titans Stadium/Week Changes/Dec_17th/metadata/dec_17/Cap3/main_images_full.json", "r"))

In [ ]:
print(len(dec_17_1))
print(len(dec_17_2))
print(len(dec_17_3))

In [ ]:
query_coords_json = []
query_coords_json_key = []
for val in dec_10:
    query_coords_json.append(dec_10[val]["position"])
    query_coords_json_key.append(val)

In [ ]:
indexed_coords_json = []
indexed_coords_key = []
dec_17 =  dec_17_1|dec_17_2|dec_17_3
for val in dec_17:
    indexed_coords_json.append(dec_17[val]["position"])
    indexed_coords_key.append(val)

In [ ]:
len(indexed_coords_json)

### Index all the coordinates in the KDTree for efficient nearest neighbor search

In [ ]:
from scipy.spatial import KDTree


# Build the tree
tree = KDTree(indexed_coords_json)

In [ ]:
matching_pairs_json = []
# Query a point (e.g., your first point)
query_idx = 0
for query_pt in query_coords_json:
    dist, index = tree.query(query_pt)
    old_image = query_coords_json_key[query_idx]
    new_image = indexed_coords_key[index]
    if dist < 1:
        if old_image == "VID_20251210_123543_00_325_IMG_00160.JPG":
            print(f"Index {query_idx} Query Image {old_image}")
            print(f"Distance: {dist} {index}")
            print(f"Nearest Image: {indexed_coords_key[index]}, index: {index}")
            print(f"query Image: {dec_10[old_image]}, Query Index: {query_idx}")
            print(f"Nearest Image search: {new_image}, {dec_17[new_image]}")
        matching_pairs_json.append(((old_image, dec_10[old_image]), (new_image, dec_17[new_image])))
    query_idx += 1

In [ ]:
len(matching_pairs_json)

In [ ]:
old_date = 'dec10_1'
new_date = 'dec17_1'
new_date_2 = 'dec17_2'
new_date_3 = 'dec17_3'

In [ ]:
def get_old_image(old_image):
    
    old_image_path = f'{RAW_DATA_DIR}/turner_images/turner_images/{old_date}/{old_image}'
    
    if os.path.exists(old_image_path):
        return old_image_path
    else:
        return None

def get_new_image(new_image):
    
    new_image_path = f'{RAW_DATA_DIR}/turner_images/turner_images/{new_date}/{new_image}'
    new_image_path_2 = f'{RAW_DATA_DIR}/turner_images/turner_images/{new_date_2}/{new_image}'
    new_image_path_3 = f'{RAW_DATA_DIR}/turner_images/turner_images/{new_date_3}/{new_image}'
    
    if os.path.exists(new_image_path):
        return new_image_path
    elif os.path.exists(new_image_path_2):
        return new_image_path_2
    elif os.path.exists(new_image_path_3):
        return new_image_path_3
    else:
        return None

### Align the frames to the default orientation

In [ ]:
import shutil
from pathlib import Path
import uuid
from groundedvision.image_alignment import align_image_files_to_standard

# Define the processed directory (convert to Path if it's a string)
processed_dir = Path(PROCESSED_DATA_DIR) / 'matched_pairs_json'
processed_dir.mkdir(parents=True, exist_ok=True)
prefix = 0
for (old, old_metadata), (new, new_metadata) in matching_pairs_json:
    print(old, new)
    print(old_metadata, new_metadata)
    geo_data_old_image_path = get_old_image(old)
    geo_data_new_image_path = get_new_image(new)
    if geo_data_old_image_path and geo_data_new_image_path:
        # Create a subfolder for each matched pair
        pair_dir = processed_dir /f"{prefix}_{str(uuid.uuid4())}"
        pair_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy the files to the processed directory
        shutil.copy2(geo_data_old_image_path, pair_dir / f'old_{old}')
        shutil.copy2(geo_data_new_image_path, pair_dir / f'new_{new}')
        print(f"  Copied pair to {pair_dir}")

        query_data = {
            'image_path': pair_dir / f'old_{old}',
            'position': old_metadata['position'],
            'rotation': old_metadata['rotation']
        }
    
        result_data = {
            'image_path': pair_dir / f'new_{new}',
            'position': new_metadata['position'],
            'rotation': new_metadata['rotation']
        }
    
        output_a, output_b = align_image_files_to_standard(
            image_path_a=pair_dir / f'old_{old}',
            rotation_a=old_metadata['rotation'],  # [pitch, yaw, roll]
            image_path_b=pair_dir / f'new_{new}',
            rotation_b=new_metadata['rotation'],
            suffix="_aligned"  # Optional, default is "_aligned"
        )
        prefix += 1
    
